In [13]:
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import time

In [20]:
# get all the thread links for the current page
def page_posts(page_url):
    response = get(page_url)
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('div', class_="california-thread-item")
    return posts

In [104]:
# scrape individual thread given a thread url
def thread_posts(thread_url):
    response = get(thread_url)
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('div', class_="bbWrapper")
    reply_count = html_soup.find('span', class_='count').text
    if 'K' in reply_count:
        reply_count = reply_count[:-1]+'000'
    pages = int(reply_count)//20 +1
    if pages>1:
        for i in range(pages-1):
            link2 = thread_url + 'page-%d' %(i+2)
            response = get(link2)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            posts += html_soup.find_all('div', class_="bbWrapper")
    return posts

# filter out unwanted text
def filer_thread_post(posts):
    docs = []
    for i in range(len(posts)):
        text = posts[i].text
        tok = 'Click to expand...'
        if tok in text:
            index =  text.find(tok)
            text = text[index+len(tok)+1:]
        text = text.replace('\n',' ').replace('...',' ').replace('..',' ')
        if len(text)*' ' != text:
            docs.append(text.replace('\n',' ').replace('...',' ').replace('..',' '))
    return docs

#get posts from a thread
def get_thread_docs(thread_url):
    posts = thread_posts(thread_url)
    docs = filer_thread_post(posts)
    #docs.append(thread_url)
    return docs

In [22]:
def main_post_info(post):
    reply_count = post.find('div', class_= 'reply-count').text.strip()
    view_count = post.find('div', class_='structItem-cell forum-view-stats view-count-cell').text.strip()
    user = post.find('a', class_="username")
    user_id = user['data-user-id']
    user_name = user.text
    user_join_date = post.find('li', class_="structItem-startDate").find('a').find('time')['datetime']

    title_head = post.find('div', class_= "structItem-title") 
    post_title = title_head.text.strip()
    post_url = 'https://www.crvownersclub.com' + title_head.find('a')['href']
    return user_id, user_name, user_join_date, post_title, reply_count, view_count, post_url 

In [255]:
t0 = time.time()
post_title = []
post_url = []
reply_count = []
view_count = []
join_date = []
user_name = []
user_id = []
for i in range(360): #360 is the total number of pages in the Problem and issue forum
    page_i_url ='https://www.crvownersclub.com/forums/problems-issues.14/page-%d' %i
    page_i_posts = page_posts(page_i_url)
    for post in page_i_posts:
        post_info = main_post_info(post)
        uid, uname, ujoin_date, title, reply, view, url = post_info
        if title[:7]=='Sticky\n':
            title = title[8:]
        post_title.append(title)
        post_url.append(url)
        reply_count.append(reply)
        view_count.append(view)
        join_date.append(ujoin_date)
        user_name.append(uname)
        user_id.append(uid)
            
print(time.time()-t0)

258.9506561756134


In [3]:
import pickle

In [256]:
# buidling data frame to store thread statistics
df_posts_p360 = pd.DataFrame({'user name': user_name,
                         'user id': user_id,
                         'user join date': join_date,
                         'post title': post_title,
                         'post url': post_url,
                         'reply count': reply_count,
                         'view count': view_count})

In [4]:
#pickle.dump( df_posts_p360, open(  "df_posts_p360.p", "wb" ) )
df_posts_p360 = pickle.load( open( "df_posts_p360.p", "rb" ) )
df_posts_p10 = pickle.load( open( "df_posts_p10.p", "rb" ) )

In [ ]:
link_list = []
df_posts = df_posts_p360
size = df_posts['post url'].size
all_docs = []
for i in range(size):
    try:
        post_url = df_posts['post url'].iloc[i]
        docs = get_thread_docs(post_url)
        docs.append(post_url)
        all_docs.append(i)
        #if i % 5000 == 0:
        #    pickle.dump( all_docs, open("all_docs_%i.p" %i, "wb" ) )
    except:
        print('there is something wroing with getting doc ', i)
        continue
    print(i+1, '/', size, len(docs)) 

In [5]:
df_posts = df_posts_p360
size = df_posts['post url'].size
all_docsX = []
for i in range(size, size):
    try:
        post_url = df_posts['post url'].iloc[i]
        post_title = df_posts['post title'].iloc[i]
        docs = get_thread_docs(post_url)
        docs.append(post_title)
        docs.append(post_url)
        all_docsC.append(docs)
    except:
        print('there is something wroing with getting doc i: ', i)
        continue
    print(i+1, '/', size, len(docs)) 
    
#pickle.dump( all_docsC, open("all_docsC.p", "wb" ) )

In [115]:
len(all_docsA)

5000

In [116]:
len(all_docsB)

5000

In [120]:
len(all_docs)

12610

In [121]:
all_docs = all_docsA + all_docsB + all_docsC
pickle.dump( all_docs, open("all_docs.p", "wb" ) )